# Credit Risk Resampling Techniques

In [45]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# Read the CSV into DataFrame

In [47]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)


df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [48]:
# Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)

# Create our target
y = df['loan_status']


In [49]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [50]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk     75036
high_risk     2500
Name: count, dtype: int64

In [51]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [52]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [53]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)


StandardScaler()

In [54]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Simple Logistic Regression

In [55]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [56]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy Score: {balanced_acc}")

Balanced Accuracy Score: 0.9905270398651461


In [57]:
# Display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")

Confusion Matrix:
[[  611     8]
 [  113 18652]]


In [58]:
# Print the imbalanced classification report
class_report = classification_report_imbalanced(y_test, y_pred)
print(f"Imbalanced Classification Report:\n{class_report}")

Imbalanced Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.98       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [59]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_ros)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [60]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_ros, y_resampled_ros)


LogisticRegression(random_state=1)

In [61]:
# Calculated the balanced accuracy score
y_pred_ros = model.predict(X_test_scaled)
balanced_acc_ros = balanced_accuracy_score(y_test, y_pred_ros)
print(f"Balanced Accuracy Score (RandomOversampler): {balanced_acc_ros}")

Balanced Accuracy Score (RandomOversampler): 0.9934649587814939


In [62]:
# Display the confusion matrix
conf_matrix_ros = confusion_matrix(y_test, y_pred_ros)
print(f"Confusion Matrix (RandomOversampler):\n{conf_matrix_ros}")

Confusion Matrix (RandomOversampler):
[[  615     4]
 [  124 18641]]


In [63]:
# Print the imbalanced classification report
class_report_ros = classification_report_imbalanced(y_test, y_pred_ros)
print(f"Imbalanced Classification Report (RandomOversampler):\n{class_report_ros}")

Imbalanced Classification Report (RandomOversampler):
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [64]:
# Resample the training data with SMOTE
smote = SMOTE(random_state=1)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_smote)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [65]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_smote, y_resampled_smote)
y_pred_smote = model.predict(X_test_scaled)


In [66]:
# Calculated the balanced accuracy score
balanced_acc_smote = balanced_accuracy_score(y_test, y_pred_smote)
print(f"Balanced Accuracy Score (SMOTE): {balanced_acc_smote}")


Balanced Accuracy Score (SMOTE): 0.9934649587814939


In [67]:
# Display the confusion matrix
conf_matrix_smote = confusion_matrix(y_test, y_pred_smote)
print(f"Confusion Matrix (SMOTE):\n{conf_matrix_smote}")

Confusion Matrix (SMOTE):
[[  615     4]
 [  124 18641]]


In [68]:
# Print the imbalanced classification report
class_report_smote = classification_report_imbalanced(y_test, y_pred_smote)
print(f"Imbalanced Classification Report (SMOTE):\n{class_report_smote}")

Imbalanced Classification Report (SMOTE):
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [69]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_resampled_rus, y_resampled_rus = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_rus)


Counter({'high_risk': 1881, 'low_risk': 1881})

In [70]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_rus, y_resampled_rus)
y_pred_cc = model.predict(X_test_scaled)


In [71]:
# Calculate the balanced accuracy score
balanced_acc_cc = balanced_accuracy_score(y_test, y_pred_cc)
print(f"Balanced Accuracy Score (ClusterCentroids): {balanced_acc_cc}")

Balanced Accuracy Score (ClusterCentroids): 0.9929503031930944


In [72]:
# Display the confusion matrix
conf_matrix_cc = confusion_matrix(y_test, y_pred_cc)
print(f"Confusion Matrix (ClusterCentroids):\n{conf_matrix_cc}")

Confusion Matrix (ClusterCentroids):
[[  614     5]
 [  113 18652]]


In [73]:
# Print the imbalanced classification report
class_report_cc = classification_report_imbalanced(y_test, y_pred_cc)
print(f"Imbalanced Classification Report (ClusterCentroids):\n{class_report_cc}")


Imbalanced Classification Report (ClusterCentroids):
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [74]:
# Resample the training data with SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 55937, 'high_risk': 55509})

In [75]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred_smoteenn = model.predict(X_test_scaled)

In [76]:
# Calculate the balanced accuracy score
balanced_acc_smoteenn = balanced_accuracy_score(y_test, y_pred_smoteenn)
print(f"Balanced Accuracy Score (SMOTEENN): {balanced_acc_smoteenn}")

Balanced Accuracy Score (SMOTEENN): 0.9934383134311076


In [77]:
# Display the confusion matrix
conf_matrix_smoteenn = confusion_matrix(y_test, y_pred_smoteenn)
print(f"Confusion Matrix (SMOTEENN):\n{conf_matrix_smoteenn}")

Confusion Matrix (SMOTEENN):
[[  615     4]
 [  125 18640]]


In [78]:
# Print the imbalanced classification report
class_report_smoteenn = classification_report_imbalanced(y_test, y_pred_smoteenn)
print(f"Imbalanced Classification Report (SMOTEENN):\n{class_report_smoteenn}")

Imbalanced Classification Report (SMOTEENN):
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

SMOTE and ROS seem to be tied somehow at 99.34649%

2. Which model had the best recall score?

They're all tied at 0.99

3. Which model had the best geometric mean score?

Perhaps I made a mistake beacuse they are all 99%